Calculate link criticality score

In [ ]:
import numpy as np  
import networkx as nx  
import xml.etree.ElementTree as ET  
from heapq import *  
from itertools import *  
import matplotlib.pyplot as plt  
import matplotlib as mpl
import xml.etree.ElementTree as ET
import networkx as nx
import traceback

def parse_sumo_net(net_xml_path):

    tree = ET.parse(net_xml_path)
    root = tree.getroot()

    G = nx.Graph()
    for node in root.findall('.//node'):
        nid = node.get('id')
        G.add_node(nid)


    edge_accum = {}  # key=(u,v)

    ignored_prefixes = [':', '-']
    for edge in root.findall('.//edge'):
        eid = edge.get('id')
        if any(eid.startswith(pref) for pref in ignored_prefixes):
            continue
        u, v = edge.get('from'), edge.get('to')
        pair = tuple(sorted([u, v]))


        length = float(edge.get('length', 0.0))
        speed  = float(edge.get('speed', 0.0))
        q      = float(edge.get('q', float('inf')))

        actual_speeds = []
        for lane in edge.findall('lane'):
            a = lane.get('actualSpeed')
            if a is not None:
                try:
                    actual_speeds.append(float(a))
                except ValueError:
                    pass

        if not actual_speeds:
            actual = speed
        else:
            actual = sum(actual_speeds) / len(actual_speeds)

        if pair not in edge_accum:
            edge_accum[pair] = {
                'length_sum': length,
                'speed_sum':  speed,
                'actual_sum': actual,
                'count':      1,
                'q_min':      q
            }
        else:
            rec = edge_accum[pair]
            rec['length_sum'] += length
            rec['speed_sum']  += speed
            rec['actual_sum'] += actual
            rec['count']     += 1
            rec['q_min'] += q
            # if q < rec['q_min']:
            #     rec['q_min'] = q

    for (u, v), rec in edge_accum.items():
        cnt = rec['count']
        G.add_edge(
            u, v,
            length      = rec['length_sum'] / cnt,
            speed       = rec['speed_sum']  / cnt,
            actualSpeed = rec['actual_sum'] / cnt,
            q           = rec['q_min']  / cnt
        )

    print(f"Total nodes: {G.number_of_nodes()}")
    print(f"Total edges (after aggregation): {G.number_of_edges()}")

    return G 

def maximum_capacity_paths(G, source, weight, target=None):  

    get_weight = lambda u, v, data: data.get(weight, 1)  
    paths = {source: [source]}  
    G_succ = G.succ if G.is_directed() else G.adj  

    push = heappush  
    pop = heappop  
    dist = {}  
    pred = {}  
    seen = {source: 0}  
    c = count()  
    fringe = []  
    push(fringe, (-1, next(c), source))  
    while fringe:  
        (d, _, v) = pop(fringe)  
        if v in dist:  
            continue  
        dist[v] = d  
        if v == target:  
            break  
        for u, e in G_succ[v].items():  
            if u == source:  
                continue  
            vu_dist = max([dist[v], -get_weight(v, u, e)])  
            if u not in seen or vu_dist < seen[u]:  
                seen[u] = vu_dist  
                push(fringe, (vu_dist, next(c), u))  
                if paths is not None:  
                    paths[u] = paths[v] + [u]  

                if dist[v] > -get_weight(v, u, e):  
                    pred[u] = pred[v]  
                else:  
                    pred[u] = (v, u)  

    dist = {i: -dist[i] for i in dist}  
    return dist, pred   

def canonical(u, v):
    return (u, v) if u <= v else (v, u)

def analyze_network_percolation(net, true_OD, total_demand):   
    net_nodes = set(map(str, net.nodes())) 
    true_OD_nodes = {tuple(map(str, od)) for od in true_OD.keys()}  


    converted_true_OD = {}  
    for (orig, dest), demand in true_OD.items():  
        orig_str = str(orig)  
        dest_str = str(dest)  
        converted_true_OD[(orig_str, dest_str)] = demand  

    unmatched_orig = set()  
    unmatched_dest = set()  
    matched_od = 0  

    for (orig, dest) in converted_true_OD.keys():  
        if orig not in net_nodes:  
            unmatched_orig.add(orig)  
        if dest not in net_nodes:  
            unmatched_dest.add(dest)  
        if orig in net_nodes and dest in net_nodes:  
            matched_od += 1  

    valid_true_OD = {  
        (orig, dest): demand   
        for (orig, dest), demand in converted_true_OD.items()   
        if orig in net_nodes and dest in net_nodes  
    }  

    criticalityScores = {}  
    counted_od = set()  
    od_result = 0  

    for Orig in net_nodes:  
        _, limitingLinksDict = maximum_capacity_paths(net, Orig, weight='q')  
        
        for Dest, raw_link in limitingLinksDict.items():  

            Dest = str(Dest)  
            
            if Orig != Dest and (Orig, Dest) in valid_true_OD:  
                link = canonical(raw_link[0], raw_link[1])   
                
                criticalityScores.setdefault(link, 0)  
                criticalityScores[link] += valid_true_OD[(Orig, Dest)] / float(total_demand)  
                od_result += valid_true_OD[(Orig, Dest)]  
                counted_od.add((Orig, Dest))  

    linkInfoList = []
    for u, v in net.edges():
        key = canonical(u, v)
        q = float(net[u][v]['q'])
        score = criticalityScores.get(key, 0)
        linkInfoList.append((u, v, q, score))

    alpha = np.sum([q * score for (_, _, q, score) in linkInfoList])

    rhos = [rho/1000.0 for rho in range(1001)]

    y_vals = [
    sum(s for (_,_,q,s) in linkInfoList if q >= rho)
    if abs(rho - 1.0) < 1e-9
    else sum(s for (_,_,q,s) in linkInfoList if q > rho)
    for rho in rhos]

    return linkInfoList, alpha, od_result


def plot_ud_alpha(net, link_info):
    rho_values = [rho/1000.0 for rho in range(1001)]

    unaffected_demand = [np.sum([link[3] for link in link_info if link[2] > rho]) for rho in rho_values]
    total_nodes = net.number_of_nodes()  

    largest_cc_sizes = []
    second_largest_cc_sizes = []
    for rho in rho_values:

        H = nx.Graph()
        H.add_nodes_from(net.nodes())
        for u, v, data in net.edges(data=True):
            if data['q'] > rho:
                H.add_edge(u, v)

        components = nx.connected_components(H)

        if components:
            comps_sorted = sorted(components, key=lambda comp: len(comp), reverse=True)
            largest_size = len(comps_sorted[0])

            second_largest_size = len(comps_sorted[1]) if len(comps_sorted) > 1 else 0
        else:
            largest_size = 0
            second_largest_size = 0
        largest_cc_sizes.append(largest_size / total_nodes)
        second_largest_cc_sizes.append(second_largest_size / total_nodes)

    
    idx_max2 = int(np.argmax(second_largest_cc_sizes))
    rho_max2 = rho_values[idx_max2]

    return rho_max2

Make network weighted matrix 

In [ ]:
########## Here are the associated DomiRank functions #############
import numpy as np
import scipy as sp
import scipy.sparse
import networkx as nx
import multiprocessing as mp
########## Here are the general functions needed for efficient dismantling and testing of networks #############

def get_largest_component(G, strong = False):
    '''
    here we get the largest component of a graph, either from scipy.sparse or from networkX.Graph datatype.
    1. The argument changes whether or not you want to find the strong or weak - connected components of the graph'''
    if type(G) == nx.classes.graph.Graph: #check if it is a networkx Graph
        if nx.is_directed(G) and strong == False:
            GMask = max(nx.weakly_connected_components(G), key = len)
        if nx.is_directed(G) and strong == True:
            GMask = max(nx.strongly_connected_components(G), key = len)
        else:
            GMask = max(nx.connected_components(G), key = len)
        G = G.subgraph(GMask)
    else:
        raise TypeError('You must input a networkx.Graph Data-Type')
    return G

def relabel_nodes(G, yield_map = False):
    '''relabels the nodes to be from 0, ... len(G).
    1. Yield_map returns an extra output as a dict. in case you want to save the hash-map to retrieve node-id'''
    if yield_map == True:
        nodes = dict(zip(range(len(G)), G.nodes()))
        G = nx.relabel_nodes(G, dict(zip(G.nodes(), range(len(G)))))
        return G, nodes
    else:
        G = nx.relabel_nodes(G, dict(zip(G.nodes(), range(len(G)))))
        return G

def get_component_size(G, strong = False):
    '''
    here we get the largest component of a graph, either from scipy.sparse or from networkX.Graph datatype.
    1. The argument changes whether or not you want to find the strong or weak - connected components of the graph'''
    if type(G) == nx.classes.graph.Graph: #check if it is a networkx Graph
        if nx.is_directed(G) and strong == False:
            GMask = max(nx.weakly_connected_components(G), key = len)
        if nx.is_directed(G) and strong == True:
            GMask = max(nx.strongly_connected_components(G), key = len)
        else:
            GMask = max(nx.connected_components(G), key = len)
        G = G.subgraph(GMask)
        return len(GMask)        
    elif type(G) == scipy.sparse.csr_array:
        if strong == False:
            connection_type = 'weak'
        else:
            connection_type = 'strong'
        noComponent, lenComponent = sp.sparse.csgraph.connected_components(G, directed = True, connection = connection_type, return_labels = True)
        return np.bincount(lenComponent).max()
    else:
        raise TypeError('You must input a networkx.Graph Data-Type or scipy.sparse.csr array')
        
def get_link_size(G):
    if type(G) == nx.classes.graph.Graph: #check if it is a networkx Graph
        links = len(G.edges()) #convert to scipy sparse if it is a graph 
    elif type(G) == scipy.sparse.csr_array:
        links = G.sum()
    else:
        raise TypeError('You must input a networkx.Graph Data-Type')
    return links

def remove_node(G, removedNode):
    '''
    removes the node from the graph by removing it from a networkx.Graph type, or zeroing the edges in array form.
    '''
    if type(G) == nx.classes.graph.Graph: #check if it is a networkx Graph
        if type(removedNode) == int:
            G.remove_node(removedNode)
        else:
            for node in removedNode:
                G.remove_node(node) #remove node in graph form
        return G
    elif type(G) == scipy.sparse.csr_array:
        diag = sp.sparse.csr_array(sp.sparse.eye(G.shape[0])) 
        diag[removedNode, removedNode] = 0 #set the rows and columns that are equal to zero in the sparse array
        G = diag @ G 
        return G @ diag
    
def generate_attack(centrality, node_map = False):
    '''we generate an attack based on a centrality measure - 
    you can possibly input the node_map to convert the attack to have the correct nodeID'''
    if node_map == False:
        node_map = range(len(centrality))
    else:
        node_map = list(node_map.values())
    zipped = dict(zip(node_map, centrality))
    attackStrategy = sorted(zipped, reverse = True, key = zipped.get)
    return attackStrategy

def generate_attack_our(centrality, node_map = False):
    '''we generate an attack based on a centrality measure - 
    you can possibly input the node_map to convert the attack to have the correct nodeID'''
    if node_map == False:
        node_map = range(len(centrality))
    else:
        node_map = list(node_map.values())
    zipped = dict(zip(node_map, centrality))
    attackStrategy = sorted(zipped, reverse = True, key = zipped.get)
    return attackStrategy

def network_attack_sampled(G, attackStrategy, sampling = 0):
    '''Attack a network in a sampled manner... recompute links and largest component after every xth node removal, according to some - 
    G: is the input graph, preferably as a sparse array.
    inputed attack strategy
    Note: if sampling is not set, it defaults to sampling every 1%, otherwise, sampling is an integer
    that is equal to the number of nodes you want to skip every time you sample. 
    So for example sampling = int(len(G)/100) would sample every 1% of the nodes removed'''
    if type(G) == nx.classes.graph.Graph: #check if it is a networkx Graph
        GAdj = nx.to_scipy_sparse_array(G) #convert to scipy sparse if it is a graph 
    else:
        GAdj = G.copy()
    
    if sampling == 0:
        if GAdj.shape[0] < 100:
            sampling = 1
        else:
            sampling = int(GAdj.shape[0]/100)
    N = GAdj.shape[0]
    initialComponent = get_component_size(GAdj)
    initialLinks = get_link_size(GAdj)
    m = GAdj.sum()/N
    componentEvolution = np.zeros((N//sampling + 1))
    linksEvolution = np.zeros((N//sampling) + 1)
    j = 0 
    for i in range(N-1):
        if i % sampling == 0:
            if i == 0:
                componentEvolution[j] = get_component_size(GAdj)/initialComponent
                linksEvolution[j] = get_link_size(GAdj)/initialLinks
                j+=1 
            else:
                GAdj = remove_node(GAdj, attackStrategy[i-sampling:i])
                componentEvolution[j] = get_component_size(GAdj)/initialComponent
                linksEvolution[j] = get_link_size(GAdj)/initialLinks
                j+=1
    return componentEvolution, linksEvolution



######## Beginning of domirank stuff! ####################

def domirank(G, analytical = True, sigma = -1, dt = 0.1, epsilon = 1e-5, maxIter = 1000, checkStep = 10):
    '''
    G is the input graph as a (preferably) sparse array.
    This solves the dynamical equation presented in the Paper: "DomiRank Centrality: revealing structural fragility of
complex networks via node dominance" and yields the following output: bool, DomiRankCentrality
    Here, sigma needs to be chosen a priori.
    dt determines the step size, usually, 0.1 is sufficiently fine for most networks (could cause issues for networks
    with an extremely high degree, but has never failed me!)
    maxIter is the depth that you are searching with in case you don't converge or diverge before that.
    Checkstep is the amount of steps that you go before checking if you have converged or diverged.
    
    
    This algorithm scales with O(m) where m is the links in your sparse array.
    '''
    if type(G) == nx.classes.graph.Graph: #check if it is a networkx Graph
        G = nx.to_scipy_sparse_array(G) #convert to scipy sparse if it is a graph 
    else:
        G = G.copy()
    if analytical == False:

        if sigma == -1:
            sigma, _ = optimal_sigma(G, analytical = False, dt=dt, epsilon=epsilon, maxIter = maxIter, checkStep = checkStep) 
        pGAdj = sigma*G.astype(np.float64)
        Psi = np.ones(pGAdj.shape[0]).astype(np.float64)/pGAdj.shape[0]
        maxVals = np.zeros(int(maxIter/checkStep)).astype(np.float64)
        dt = np.float64(dt)
        j = 0
        boundary = epsilon*pGAdj.shape[0]*dt
        for i in range(maxIter):
            tempVal = ((pGAdj @ (1-Psi)) - Psi)*dt
            Psi += tempVal.real
            if i% checkStep == 0:
                if np.abs(tempVal).sum() < boundary:
                    break
                maxVals[j] = tempVal.max()
                if i == 0:
                    initialChange = maxVals[j]
                if j > 0:
                    if maxVals[j] > maxVals[j-1] and maxVals[j-1] > maxVals[j-2]:
                        return False, Psi
                j+=1

        return True, Psi
    else:

        if sigma == -1:
            sigma = optimal_sigma(G, analytical = True, dt=dt, epsilon=epsilon, maxIter = maxIter, checkStep = checkStep) 
        Psi = sp.sparse.linalg.spsolve(sigma*G + sp.sparse.identity(G.shape[0]), sigma*G.sum(axis=-1))
        return True, Psi
    
def find_eigenvalue(G, minVal = 0, maxVal = 1, maxDepth = 100, dt = 0.1, epsilon = 1e-5, maxIter = 100, checkStep = 10):
    '''
    G: is the input graph as a sparse array.
    Finds the largest negative eigenvalue of an adjacency matrix using the DomiRank algorithm.
    Currently this function is only single-threaded, as the bisection algorithm only allows for single-threaded
    exection. Note, that this algorithm is slightly different, as it uses the fact that DomiRank diverges
    at values larger than -1/lambN to its benefit, and thus, it is not exactly bisection theorem. I haven't
    tested in order to see which exact value is the fastest for execution, but that will be done soon!
    Some notes:
    Increase maxDepth for increased accuracy.
    Increase maxIter if DomiRank doesn't start diverging within 100 iterations -- i.e. increase at the expense of 
    increased computational cost if you want potential increased accuracy.
    Decrease checkstep for increased error-finding for the values of sigma that are too large, but higher compcost
    if you are frequently less than the value (but negligible compcost).
    '''
    x = (minVal + maxVal)/G.sum(axis=-1).max()
    minValStored = 0
    for i in range(maxDepth):
        if maxVal - minVal < epsilon:
            break
        if domirank(G, False, x, dt, epsilon, maxIter, checkStep)[0]:
            minVal = x
            x = (minVal + maxVal)/2
            minValStored = minVal
        else:
            maxVal = (x + maxVal)/2
            x = (minVal + maxVal)/2
        if minVal == 0:
            print(f'Current Interval : [-inf, -{1/maxVal}]')
        else:
            print(f'Current Interval : [-{1/minVal}, -{1/maxVal}]')
    finalVal = (maxVal + minVal)/2
    return -1/finalVal



############## This section is for finding the optimal sigma #######################
def process_iteration_wrapper(i, sigma, analytical, spArray, maxIter, checkStep, dt, epsilon, sampling):

    tf, domiDist = domirank(spArray, analytical=analytical, sigma=sigma, dt=dt, epsilon=epsilon, maxIter=maxIter, checkStep=checkStep)

    domiAttack = generate_attack(domiDist)

    ourTempAttack, _ = network_attack_sampled(spArray, domiAttack, sampling=sampling)
    finalErrors = ourTempAttack.sum()
    return (i, finalErrors)

def optimal_sigma(spArray, analytical=True, endVal=0, startval=1e-6, iterationNo=100,
                  dt=0.1, epsilon=1e-5, maxIter=100, checkStep=10, maxDepth=100, sampling=0):
    
    from multiprocessing.dummy import Pool as ThreadPool

    if endVal == 0:
        endVal = find_eigenvalue(spArray, maxDepth=maxDepth, dt=dt, epsilon=epsilon, maxIter=maxIter, checkStep=checkStep)

    endval = -0.9999 / endVal

    tempRange = np.linspace(startval, endval, iterationNo)
    

    args = [(i, sigma, analytical, spArray, maxIter, checkStep, dt, epsilon, sampling)
            for i, sigma in enumerate(tempRange)]

    with ThreadPool() as pool:
        results = pool.starmap(process_iteration_wrapper, args)

    results.sort(key=lambda x: x[0])
    finalErrors = np.array([r[1] for r in results])
    best_index = np.argmin(finalErrors)
    best_sigma = tempRange[best_index]
    return best_sigma, finalErrors, tempRange

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

def network_attack_metrics(linkInfoList, attackStrategy, sampling=0):

    N = len(attackStrategy)
    if sampling == 0:
        sampling = 1 if N < 100 else max(1, N // 100)


    total_initial = sum(score for _, _, _, score in linkInfoList)

    G0 = nx.Graph()
    for u, v, _, _ in linkInfoList:
        G0.add_edge(u, v)
    initial_lcc = len(max(nx.connected_components(G0), key=len))

    num_samples = N // sampling + 1
    ua_curve = np.zeros(num_samples, dtype=float)
    lcc_curve = np.zeros(num_samples, dtype=float)
    frac_removed = np.zeros(num_samples, dtype=float)

    removed = set()
    idx = 0

    ua_curve[idx] = total_initial
    lcc_curve[idx] = initial_lcc
    frac_removed[idx] = 0.0
    idx += 1


    for i, node in enumerate(attackStrategy, start=1):
        removed.add(node)
        if i % sampling == 0:
 
            ua = sum(score for u, v, _, score in linkInfoList
                     if u not in removed and v not in removed)
            ua_curve[idx] = ua

            G = G0.copy()
            G.remove_nodes_from(removed)
            if G.number_of_nodes() > 0:
                lcc_curve[idx] = len(max(nx.connected_components(G), key=len))
            else:
                lcc_curve[idx] = 0
   
            frac_removed[idx] = i / float(N)
            idx += 1


    ua_curve = ua_curve[:idx]
    lcc_curve = lcc_curve[:idx]
    frac_removed = frac_removed[:idx]

    return frac_removed, ua_curve, lcc_curve
def resilience_attack_metrics(linkInfoList, attackStrategy, sampling=0):
 
    N = len(attackStrategy)
    if sampling == 0:
        sampling = 1 if N < 100 else max(1, N // 100)

    total_initial = sum(q*score for _, _, q, score in linkInfoList)

    G0 = nx.Graph()
    for u, v, _, _ in linkInfoList:
        G0.add_edge(u, v)
    initial_lcc = len(max(nx.connected_components(G0), key=len))

    num_samples = N // sampling + 1
    resilience_curve = np.zeros(num_samples, dtype=float)
    lcc_curve = np.zeros(num_samples, dtype=float)
    frac_removed = np.zeros(num_samples, dtype=float)

    removed = set()
    idx = 0


    resilience_curve[idx] = total_initial
    lcc_curve[idx] = initial_lcc
    frac_removed[idx] = 0.0
    idx += 1


    for i, node in enumerate(attackStrategy, start=1):
        removed.add(node)
        if i % sampling == 0:
    
            re = sum(q*score for u, v, q, score in linkInfoList
                     if u not in removed and v not in removed)
            resilience_curve[idx] = re
        
            G = G0.copy()
            G.remove_nodes_from(removed)
            if G.number_of_nodes() > 0:
                lcc_curve[idx] = len(max(nx.connected_components(G), key=len))
            else:
                lcc_curve[idx] = 0
     
            frac_removed[idx] = i / float(N)
            idx += 1


    resilience_curve = resilience_curve[:idx]
    lcc_curve = lcc_curve[:idx]
    frac_removed = frac_removed[:idx]

    return frac_removed, resilience_curve, lcc_curve

import numpy as np

def network_attack_unaffected(linkInfoList, attackStrategy, sampling=0):

    N = len(attackStrategy)

    if sampling == 0:
        sampling = 1 if N < 100 else N // 100


    total_initial = sum(score for _, _, _, score in linkInfoList)

    num_samples = N // sampling + 1
    unaffected = np.zeros(num_samples, dtype=float)

    removed = set()
    idx = 0


    unaffected[idx] = total_initial
    idx += 1


    for i, node in enumerate(attackStrategy, start=1):
        removed.add(node)

  
        if i % sampling == 0:
            s = 0.0
          
            for u, v, _, score in linkInfoList:
                if u not in removed and v not in removed:
                    s += score
            unaffected[idx] = s
            idx += 1

    return unaffected


Identification of resilient-node 

In [ ]:
import time
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from scipy.sparse import csr_array

    
def compute_domirank(GAdj, net, directed=False, analytical=False,
                     max_iter=1000, dt=0.01, check_step=25):

    
    analytical = False #if you want to use the analytical method or the recursive definition
    directed = False
    seed = 42
    np.random.seed(seed)
    
    ##### #RANDOMIZATION ######
    
    #for random results
    seed = np.random.randint(0, high = 2**30-1)
    
    #for deterministic results
    #seed = 42
    
    #setting the random seed
    np.random.seed(seed)
    
    
    ##### END OF RANDOMIZATION #####
    
    ############## IMPORTANT!!!! Here you can create whatever graph you want and just comment this erdos-renyi network out ############
    #G = nx.fast_gnp_random_graph(N, 2*m/N, seed = seed, directed = directed) #####THIS IS THE INPUT, CHANGE THIS TO ANY GRAPH #######

    
    N = len(net)

    if directed:
        GAdj = sp.sparse.csr_array(GAdj.T)


    G, node_map = relabel_nodes(net, yield_map=True)

    t1 = time.time()
    lambN = find_eigenvalue(GAdj, maxIter=max_iter, dt=dt, checkStep=check_step)
    t2 = time.time()
    print(f"\nFound smallest eigenvalue λ_N = {lambN:.6f} (took {t2-t1:.2f}s)")


    sigma, sigma_array, sigma_area = optimal_sigma(
        GAdj, analytical=analytical, endVal=lambN
    )
    print(f"Optimal σ = {sigma * -lambN:.6f} / -λ_N")

    _, domi_dist = domirank(GAdj, analytical=analytical, sigma=sigma)

    return sigma, sigma_array, sigma_area, domi_dist, node_map

def compute_attack_metrics(link_info, domi_dist, node_map):
 
    attack_strategy = generate_attack(domi_dist, node_map)
    rho, ua_curve, lcc_curve = resilience_attack_metrics(link_info, attack_strategy)


    ua_frac = ua_curve / ua_curve[0]
    lcc_frac = lcc_curve / lcc_curve[0]


    area_ud  = np.trapz(ua_frac, rho)
    area_lcc = np.trapz(lcc_frac, rho)
   
    return rho, ua_curve, lcc_curve, area_ud, area_lcc, attack_strategy

def plot_domirank_curve(sigma_area, sigma_array, sigma):

    idx = np.where(sigma_array == sigma_array.min())[0][-1]

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.plot(sigma_area, sigma_array, '-k', label='Area curve')
    ax.plot(
        sigma_area[idx],
        sigma_array[idx],
        'ro',
        mfc='none',
        markersize=8,
        label=rf'Opt $\sigma$={sigma:.3f}'
    )
    ax.set_xlabel(r'$\sigma$')
    ax.set_ylabel('Area under LCC curve')
    ax.legend()
    fig.tight_layout()
    plt.show()

def plot_attack_curves(rho, ua_curve, lcc_curve):

    ua_frac  = ua_curve  / ua_curve[0]
    lcc_frac = lcc_curve / lcc_curve[0]

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))

    ax1.plot(rho, ua_frac, '-o')
    ax1.set_ylabel('Unaffected Demand (fraction)')
    ax1.grid(True)

    ax2.plot(rho, lcc_frac, '-o', color='orange')
    ax2.set_xlabel('Fraction of nodes removed')
    ax2.set_ylabel('LCC size (fraction)')
    ax2.grid(True)

    fig.tight_layout()
    plt.show()

def save_results_to_csv(a_list, ud_auc, lcc_auc, knee_a, filename):

    a_list = np.array(a_list)
    ud_auc = np.array(ud_auc)
    lcc_auc = np.array(lcc_auc)
    knee_a = np.array(knee_a)

    df = pd.DataFrame({
        'a_list': a_list,
        'ud_auc': ud_auc,
        'lcc_auc': lcc_auc,
        'knee_a':knee_a
    })
    

    df.to_csv(filename, index=False)
    
    return df, filename
def save_curves_to_csv(rho, ua_curve, lcc_curve, filename):

    df = pd.DataFrame({
        'rho':rho,
        'ua_curve': ua_curve,
        'lcc_curve': lcc_curve
    })
    df.to_csv(filename, index=False)


    
def save_attack_strategy_to_csv(attack_strategy, filename):

    attack_strategy = np.array(attack_strategy)

    df = pd.DataFrame({
        'attack_strategy': attack_strategy
    }) 

    df.to_csv(filename, index=False)

def load_od_counts(input_file='node_od_counts.csv'):

    import pandas as pd
    
    try:

        od_df = pd.read_csv(input_file)
        

        od_counts = {
            (row['from_node'], row['to_node']): row['count'] 
            for _, row in od_df.iterrows()
        }    
        total_trips = sum(od_counts.values())      
        return od_counts


def load_curves_from_csv(filename):
    df = pd.read_csv(filename)
    rho = df['rho'].to_numpy()
    ua_curve = df['ua_curve'].to_numpy()
    lcc_curve = df['lcc_curve'].to_numpy()
    return ua_curve, lcc_curve

def load_attack_strategy_from_csv(filename):

    df = pd.read_csv(filename)
    attack_strategy = df['attack_strategy'].tolist()

    return attack_strategy

def process_node_dominance_and_od(link_info, ourDomiRankAttack, ourDomiRankDistribution, node_map, df_od):

    matched_keys = []
    for id_to_match in ourDomiRankAttack:  
        matched_key = [key for key, value in node_map.items() if value == id_to_match]  
        
        if matched_key:  
            matched_keys.append(matched_key[0])  
    domirank_dict = {node: ourDomiRankDistribution[node] for node in matched_keys}
    
    domi_df = pd.DataFrame(list(domirank_dict.items()), columns=["node_id", "DomiRank"])
    domi_df['node_name'] = domi_df['node_id'].map(node_map)
    link_df = pd.DataFrame(link_info, columns=['from','to','q','score'])
    endpoint_scores = pd.concat([
        link_df[['from','score']].rename(columns={'from':'node'}),
        link_df[['to'  ,'score']].rename(columns={'to'  :'node'})
    ], ignore_index=True) \
     .groupby('node', as_index=True)['score'] \
     .sum()
    domi_df['node_score'] = domi_df['node_name'].astype(str).map(endpoint_scores).fillna(0)

    domi_df.sort_values(by='DomiRank', ascending=False, inplace=True)

    df_od['id'] = df_od['id'].astype(str)
    domi_merged_df = domi_df.merge(df_od, left_on='node_name', right_on='id', how='left')
    
    return domi_merged_df
    
def main(GAdj, net, link_info, filename_curve, filename_attack, filename_dfod, df_od, directed=False, analytical=False):
    sigma, sigma_array, sigma_area, domi_dist, node_map = compute_domirank(
        GAdj, net, directed, analytical
    )
    plot_domirank_curve(sigma_area, sigma_array, sigma)

    rho, ua_curve, lcc_curve, area_ud, area_lcc, attack_strategy = compute_attack_metrics(
        link_info, domi_dist, node_map
    )
    plot_attack_curves(rho, ua_curve, lcc_curve)
    domi_merged_df = process_node_dominance_and_od(link_info, attack_strategy, domi_dist, node_map, df_od)
    domi_merged_df.to_csv(filename_dfod, index=False)
    
    save_curves_to_csv(rho, ua_curve, lcc_curve, filename_curve)

    save_attack_strategy_to_csv(attack_strategy, filename_attack)

    return sigma, area_ud, area_lcc

Main 

In [ ]:
import numpy as np
import networkx as nx
import scipy as sp
import scipy.sparse
from tqdm import tqdm

time_periods = ['6-7', '7-8', '8-9', '9-10', '10-11', '11-12', 
                '12-13', '13-14', '14-15', '15-16', '16-17', 
                '17-18', '18-19', '19-20', '20-21', '21-22']

base_path = r'E:\SUMO\Paper1-data\research_data\29May_Thur'

# 固定随机种子
seed = 42
np.random.seed(seed)
analytical = False #if you want to use the analytical method or the recursive definition
directed = False

result_dict = {
    'alpha':[],
    'rho':[],
    'ud': [],
    'lcc': [],
    'knee': [],
    'sigma': []
}
for period in time_periods:
    net_xml_path = f'{base_path}\\{period}\\enhenced_topology.net.xml'
    net = parse_sumo_net(net_xml_path)
    od_counts_path = f'{base_path}\\{period}\\node_od_counts.csv'
    od_counts = load_od_counts(od_counts_path)
    total_count = len(od_counts)
    
    df_od = od_counts_to_dataframe(od_counts)
    # 2.2 读取knee_a
    pareto_csv = os.path.join(base_path, period, 'pareto_param.csv')
    if not os.path.isfile(pareto_csv):
        raise FileNotFoundError(f"Missing file: {pareto_csv}")
    pareto_df = pd.read_csv(pareto_csv)

    link_info, alpha, missing, od_result = analyze_network_percolation(net, od_counts, total_count)
    
    print("Top 10 Critical Links:")
    top_links = sorted(link_info, key=lambda x: x[3], reverse=True)[:10]
    for link in top_links:
        print(f"Link {link[0]}->{link[1]}: q={link[2]:.4f}, Criticality={link[3]:.4f}")
    
    rho = plot_ud_alpha(net, link_info)
    node_list = list(net.nodes())
    A = nx.to_scipy_sparse_array(net)

    rows, cols, data = [], [], []
    for u, v, q, score in link_info:
        if score > 0:
            i = node_list.index(u)
            j = node_list.index(v)
            rows.extend([i, j])
            cols.extend([j, i])
            data.extend([score*q, score*q])
    
    D = scipy.sparse.coo_matrix(
        (data, (rows, cols)),
        shape=(len(node_list), len(node_list))
    )
    
    if D.data.size > 0:
        max_score = D.data.max()
        D_norm = D.multiply(1.0 / max_score)
    else:
        D_norm = D.copy()
    as_range = np.linspace(0, 1, 51)
    results = []
    
    for a in tqdm(as_range, desc=f"Grid Search {period}", unit="step"):
        res = evaluate_weights_ab(a,
                                  link_info,
                                  D_norm,
                                  A,
                                  net,
                                  sampling=0)
        results.append(res)
    
    pareto_front = non_dominated_sort(results)
    
    print("\nPareto-optimal (a, b) with (UD_auc, LCC_auc):")
    for a, b, ua, lcc in sorted(pareto_front, key=lambda x: x[0]):
        print(f"a={a:.2f}, b={b:.2f}  →  UD_auc={ua:.4f}, LCC_auc={lcc:.4f}")
    
    a_list, ud_auc, lcc_auc = [], [], []
    for a, b, ua, lcc in sorted(pareto_front, key=lambda x: x[0]):
        a_list.append(a)
        ud_auc.append(ua)
        lcc_auc.append(lcc)
    
    a_list = np.array(a_list)
    ud_auc = np.array(ud_auc)
    lcc_auc = np.array(lcc_auc)
    
    knee_a = plot_pareto_knee(ud_auc, lcc_auc, a_list)
    based_path = r'E:\SUMO\Paper1-data\research_data\29May_Thur_revison'
    df_parato, saved_file = save_results_to_csv(
        a_list, ud_auc, lcc_auc, knee_a,
        f'{based_path}\\{period}\\pareto_param.csv'
    )
    

    GAdj_acces = nx.to_scipy_sparse_array(net)
    W_sp = A*knee_a + D_norm*(1-knee_a)
    GAdj = W_sp
    GAdj_quanlity = csr_array(D_norm)
    

    filename_curve0 = f'{based_path}\\{period}\\all_curve_param.csv'
    filename_attack0 = f'{based_path}\\{period}\\all_attackstrategy_param.csv'
    filename_dfod0 = f'{based_path}\\{period}\\all_domi_mergerd_OD.csv'
    sigma0, area_ud0, area_lcc0 = main(GAdj, net, link_info, filename_curve0, filename_attack0, filename_dfod0, df_od, directed=False, analytical=False)
    
    filename_curve1 = f'{based_path}\\{period}\\quanlity_curve_param.csv'
    filename_attack1 = f'{based_path}\\{period}\\quanlity_attackstrategy_param.csv'
    filename_dfod1 = f'{based_path}\\{period}\\quanlity_domi_mergerd_OD.csv'
    sigma1,area_ud1, area_lcc1 = main(GAdj_quanlity, net, link_info, filename_curve1, filename_attack1, filename_dfod1, df_od, directed=False, analytical=False)
    
    filename_curve2 = f'{based_path}\\{period}\\curve_param.csv'
    filename_attack2 = f'{based_path}\\{period}\\attackstrategy_param.csv'
    filename_dfod2 = f'{based_path}\\{period}\\domi_mergerd_OD.csv'
    sigma2,area_ud2, area_lcc2 = main(GAdj_acces, net, link_info, filename_curve2, filename_attack2, filename_dfod2, df_od, directed=False, analytical=False)

    result_dict['ud'].append(area_ud0)
    result_dict['lcc'].append(area_lcc0)
    result_dict['knee'].append(knee_a)
    result_dict['sigma'].append(sigma0)
    result_dict['alpha'].append(alpha) 
    result_dict['rho'].append(rho) 

result_dict_df = pd.DataFrame(result_dict)
filename_all = f'{based_path}\\result_dict_df.csv'
result_dict_df.to_csv(filename_all, index=False, encoding='utf-8-sig')

Resilience Linear Gain Interval 

In [ ]:
import os
import pandas as pd
import numpy as np

from heapq import heappush, heappop
from itertools import count
from collections import defaultdict


def analyze_top10_resilience_improvement(net, true_OD, total_demand, output_csv):

    converted_true_OD = {(str(o), str(d)): f for (o, d), f in true_OD.items()}
    valid_true_OD = {od: f for od, f in converted_true_OD.items() if od[0] in net and od[1] in net}

    crit0 = compute_criticality(net, valid_true_OD, total_demand)

    top_links = sorted(crit0.items(), key=lambda x: x[1], reverse=True)[:5]

    all_records = []
    for rank, (link, s0) in enumerate(top_links, start=1):
        u0, v0 = link
        q0 = net[u0][v0]['q']

        second_vals = []
        for Orig in set(o for o, _ in valid_true_OD.keys()):
            _, pred, paths = maximum_capacity_paths(net, Orig, weight='q')
            for od, f in valid_true_OD.items():
                if Orig != od[0] or od[1] not in pred: continue
                if canonical(*pred[od[1]]) != link: continue
                qs = [net[a][b]['q'] for a, b in zip(paths[od[1]], paths[od[1]][1:])]
                uniq = sorted(set(qs))
                if len(uniq) >= 2 and uniq[1] != q0:
                    second_vals.append(uniq[1])

        if second_vals:
            bound_min = min(second_vals)
            delta = bound_min - q0
            q2_list = [bound_min - 1e-7] + [bound_min + delta * m for m in (1,3,5,7,10)]
        else:
            q2_list = [q0]
  
        for q2 in q2_list:
            q2_val = max(q0, q2)
         
            new_q = {canonical(u,v): data['q'] for u,v,data in net.edges(data=True)}
            new_q[link] = q2_val
          
            net2 = net.copy()
            for (u,v), qq in new_q.items(): net2[u][v]['q'] = qq
        
            crit2 = compute_criticality(net2, valid_true_OD, total_demand)
            # alpha0与alpha2
            alpha0 = sum(net[u][v]['q'] * crit0.get(canonical(u,v),0) for u,v in net.edges())
            alpha2 = sum(
                new_q.get(canonical(u,v), net[u][v]['q']) * crit2.get(canonical(u,v), 0)
                for u, v in net.edges()
            )
            s_new = crit2.get(link, 0.0)
 
            s_old = crit0[link]

            all_records.append({
                'rank': rank,
                'link': link,
                'original_q': q0,
                'new_q': q2_val,
                'delta_q': q2_val - q0,
                'alpha0': alpha0,
                'alpha2': alpha2,
                'delta_alpha': alpha2 - alpha0,
                'gain_rate': (alpha2 - alpha0)/(q2_val - q0) if q2_val>q0 else 0.0,
                's_old':s_old,
                's_new':s_new
            })

        all_records.append({})

    df = pd.DataFrame(all_records)
    df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"Results written to {output_csv}")
